In [ ]:
import os
base_path = os.path.expanduser("/Users/mariannelado-roy/data_grid2op")

# Parameters

In [ ]:
from grid2op.Reward import L2RPNReward
from stable_baselines3 import PPO, DQN, A2C
from sb3_contrib import ARS, QRDQN, TRPO, RecurrentPPO

### REWARDS #############################
reward_function=L2RPNReward
prefix='L2RPNReward_Simple'
#########################################

########## TRAINING PARAMETERS #################### 
N_TIMESTEP = 1_000_000
N_EVAL_EPISODES= 5
EVAL_FREQ = 30_000
algorithms = [PPO, RecurrentPPO, TRPO, A2C, DQN, QRDQN, ARS]
################################################### 


####### ENVIRONNEMENTS ###########################
env_name = "l2rpn_case14_sandbox"
env_config = {
    "backend_options": {},
    "env_name": "l2rpn_case14_sandbox", 
    "env_is_test": False,
    "act_type": "discrete", 
    "act_attr_to_keep": ["set_line_status_simple", "set_bus"], 
    "reward_class": reward_function, 
}
################################################### 

In [6]:
def fpath(algo,prefix=''):
    model_name = algo.__name__
    tb_log_name = prefix + "_"+ f"{model_name}"
    model_path = "./" + prefix + "/" f"{model_name}"
    return model_name, model_path, tb_log_name

# Dowload environnement
#### Split in training / validation set

In [ ]:
import grid2op
from env_wrapper import Grid2opEnvWrapper

train_dir = os.path.join(base_path, f"{env_name}_train/")
val_dir = os.path.join(base_path, f"{env_name}_val/")
test_dir = os.path.join(base_path, f"{env_name}_test/")

def directories_exist(train_path, val_path, test_path):
    return os.path.exists(train_path) and os.path.exists(val_path) and os.path.exists(test_path)

# Make environment
env = grid2op.make(env_name)

if directories_exist(train_dir, val_dir, test_dir):
    print("Directories already exist. Skipping train-val-test split.")
else:
    print("Directories do not exist. Proceeding with train-val-test split.")
    # Perform the split
    try:
        nm_env_train, nm_env_val, nm_env_test = env.train_val_split_random(
            pct_val=10, # 10% validation
            pct_test=10, # 10% test
            add_for_train="train",
            add_for_val="val",
            add_for_test="test"
        )
        print(f"Training environment created: {nm_env_train}")
        print(f"Validation environment created: {nm_env_val}")
        print(f"Test environment created: {nm_env_test}")
    except OSError as e:
        print(f"An error occurred during splitting: {e}")


Directories already exist. Skipping train-val-test split.



# Create the environments


In [ ]:
# Training environnement
train_env_config = env_config.copy()
train_env_config["data_set"] = "train"
train_env = Grid2opEnvWrapper(train_env_config)

# Validation environnement
val_env_config = env_config.copy()
val_env_config["data_set"] = "val"
val_env = Grid2opEnvWrapper(val_env_config)

# Training

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor


monitor_env = Monitor(val_env, filename='./monitor_log', allow_early_resets=True)
log_dir = "./tensorboard_logs/"

# Loop over each algorithm
for algo in algorithms:
    # Define the model path and tensorboard log name dynamically
    model_name, model_path, tb_log_name = fpath(algo,prefix)
    
    # Create Call back:
    eval_callback = EvalCallback( 
        eval_env=monitor_env, 
        n_eval_episodes=N_EVAL_EPISODES,
        eval_freq=EVAL_FREQ, 
        log_path = model_path, 
        best_model_save_path=model_path,
        deterministic=True, 
        render=False,
        verbose=0 )


    # Initialize the algorithm with custom hyperparameters
    policy = "MlpPolicy"
    if algo == RecurrentPPO:
        policy =  "MlpLstmPolicy"

    sb3_algo = algo(
        policy,
        train_env,
        verbose=0,
        tensorboard_log=log_dir,
    )

    # Train the model
    sb3_algo.learn(
        total_timesteps=N_TIMESTEP, 
        tb_log_name=tb_log_name,
        callback=eval_callback
    )

    # Save the model
    sb3_algo.save(model_path+'/'+model_name)

    print(f"{model_name} model saved at {model_path}")


In [ ]:
# IF INTERRUPT

# Save the model
# sb3_algo.save(model_path+'/'+model_name)

# print(f"{model_name} model saved at {model_path}")

# algorithms=[A2C]

#### Look at evaluations

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt


# # Loop over each trained algorithm
# for algo in algorithms:
#     _, model_path, _ =  fpath(algo,prefix)

#     data = np.load(model_path+'/evaluations.npz')

#     results = np.mean(data['results'],axis=1)
#     ep_len = np.mean(data['ep_lengths'],axis=1)
#     t = data['timesteps']

#     plt.figure()
#     plt.plot(t,results/100, label='rewards/100')
#     plt.plot(t,ep_len,label='ep_len')
#     plt.legend()

# Test set

### Compare to baseline agent

In [ ]:
from grid2op.Runner import Runner
from grid2op.Agent import DoNothingAgent, TopologyGreedy, PowerLineSwitch, RandomAgent
from agent_wrapper import Grid2opAgentWrapper
import numpy as np


# Testing parameters ####################
nb_episode_test = 50
########################################

# Environnement
# Test environnement
test_env_config = env_config.copy()
test_env_config["data_set"] = "test"
test_env_config["reward_class"] = L2RPNReward # Always compare with the L2PRN reward
test_env = Grid2opEnvWrapper(test_env_config)

seeds_test_env = tuple(range(nb_episode_test))  # Seeds for the environment
seeds_test_agent = tuple(range(nb_episode_test))  # Seeds for the agent
ts_ep_test = tuple(range(nb_episode_test)) 

# Trained agents
for algo in algorithms:
    # Define the model path and tensorboard log name dynamically
    model_name, model_path, _ =  fpath(algo,prefix)

    # Load the trained model
    sb3_algo_to_test = algo.load(model_path+'/'+model_name, env=test_env)
    # sb3_algo_to_test = algo.load(model_path+'/best_model', env=test_env)

    # Convert to grid2op agent
    my_agent = Grid2opAgentWrapper(test_env, sb3_algo_to_test)

    runner = Runner(**test_env._g2op_env.get_params_for_runner(),
                    agentClass=None,
                    agentInstance=my_agent)

    res = runner.run(nb_episode=nb_episode_test,
                    env_seeds=seeds_test_env,
                    agent_seeds=seeds_test_agent,
                    episode_id=ts_ep_test,
                    add_detailed_output=True,
                    path_save="test/" + prefix + '_' + model_name 
                    )

# # Baseline agents:
baseline_agents= [RandomAgent, DoNothingAgent]
for baseline_agent in baseline_agents : 
    
    model_name, model_path, _ =  fpath(baseline_agent,prefix)
    
    runner = Runner(**test_env._g2op_env.get_params_for_runner(),
                agentClass=baseline_agent)

    res = runner.run(nb_episode=nb_episode_test,
                 env_seeds=seeds_test_env,
                 agent_seeds=seeds_test_agent,
                 episode_id=ts_ep_test,
                 add_detailed_output=True,
                 path_save="test/" + model_name
                 )

# Look at the rewards of some episodes

In [ ]:
import matplotlib.pyplot as plt
import glob


fig, ax = plt.subplots(3,1,sharex=True)
# plt.title('Comparison of Trained agents and Baslines')
ax[1].set_xlabel('Episode #')
ax[0].set_ylabel('Cumulative reward')

reward_dict = dict()
ep_dict = dict()

for algo in algorithms:
    model_name, model_path, _ =  fpath(algo,prefix)
    files = glob.glob("test/" + prefix + '_' + model_name + '/*/rewards*')
    n_ep = np.arange(len(files))
    ep_length=[]
    rew_per_ep = []
    for file in files: # 1 file = 1 episode
        rewards = np.load( file)['data']
        rew_per_ep.append(np.nansum(rewards))
        ep_length.append( np.sum(~np.isnan(rewards))  )
    ep_dict[model_name] = np.array(ep_length)
    reward_dict[model_name] = np.array(rew_per_ep)
    ax[0].plot(n_ep, rew_per_ep, '-',label=model_name)
    ax[1].plot(n_ep, ep_length, '-',label=model_name)

ax[0].legend()

for baseline_agent in baseline_agents : 
    model_name, model_path, _ =  fpath(baseline_agent,prefix)
    files = glob.glob("test/" + model_name + '/*/rewards*')
    n_ep = np.arange(len(files))
    ep_length=[]
    rew_per_ep = []
    for file in files: # 1 file = 1 episode
        rewards = np.load( file)['data']
        rew_per_ep.append(np.nansum(rewards))
        ep_length.append( np.sum(~np.isnan(rewards))  )
    ep_dict[model_name] = np.array(ep_length)
    reward_dict[model_name] = np.array(rew_per_ep)
    ax[0].plot(n_ep, rew_per_ep, '-',label=model_name)
    ax[1].plot(n_ep, ep_length, '-',label=model_name)
ax[0].legend()

ax[2].plot(n_ep, reward_dict['TRPO'] - reward_dict['DoNothingAgent'])
ax[2].plot(n_ep, reward_dict['A2C'] - reward_dict['DoNothingAgent'])
ax[2].plot(n_ep, reward_dict['RecurrentPPO'] - reward_dict['DoNothingAgent'])

# ax[2].plot(n_ep, ep_dict['TRPO'] - ep_dict['DoNothingAgent'])
# ax[2].plot(n_ep, ep_dict['A2C'] - ep_dict['DoNothingAgent'])
# ax[2].plot(n_ep, ep_dict['RecurrentPPO'] - ep_dict['DoNothingAgent'])

plt.tight_layout()
